In [1]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input
import keras
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam as adam
import csv
import pandas
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from skimage.exposure import equalize_adapthist as clahe
import os
from keras.layers import Lambda
from scipy.misc import imresize as imresize
from scipy.misc import imread as imread

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [ ]:
# Get Steering Data:
colnames = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

data2 = pandas.read_csv('data/driving_log.csv', names=colnames)

centre_images2 = data2.center.tolist()
left_images2   = data2.left.tolist()
right_images2  = data2.right.tolist()

# Remove the first element:
centre_images2 = centre_images2[1:]
left_images2   = left_images2[1:]
right_images2  = right_images2[1:]

In [3]:
colnames = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

data1 = pandas.read_csv('driving_log.csv', names=colnames)

centre_images1 = data1.center.tolist()
left_images1   = data1.left.tolist()
right_images1  = data1.right.tolist()

# Remove the first element:
centre_images1 = centre_images1[1:]
left_images1   = left_images1[1:]
right_images1  = right_images1[1:]

In [4]:
# Add the elements to y value:

y_train1 = data1.steering.tolist()
y_train1 = y_train1[1:] # Remove the first element
y_train1 = np.float32(y_train1) # Convert to float
y_train1 =  y_train1
y_train_all1 = []

for i in range(len(centre_images1)):
    steering_angle_centre = y_train1[i]
    
    delta_steering_ang   = 0.05
    
    steering_angle_left  = steering_angle_centre + delta_steering_ang
    steering_angle_right = steering_angle_centre - delta_steering_ang 
    
    y_train_all1.append(steering_angle_centre)
    y_train_all1.append(steering_angle_left)
    y_train_all1.append(steering_angle_right)

y_train_all1 = np.array(y_train_all1)

In [5]:
print(y_train_all1.shape)

(22860,)


In [ ]:
# Add the elements to y value:

y_train2 = data2.steering.tolist()
y_train2 = y_train2[1:] # Remove the first element
y_train2 = np.float32(y_train2) # Convert to float

y_train_all2 = []

for i in range(y_train2.shape[0]):
    steering_angle_centre = y_train2[i]
    
    delta_steering_ang   = 0.05
    
    steering_angle_left  = steering_angle_centre + delta_steering_ang
    steering_angle_right = steering_angle_centre - delta_steering_ang 
    
    y_train_all2.append(steering_angle_centre)
    y_train_all2.append(steering_angle_left)
    y_train_all2.append(steering_angle_right)

y_train_all2 = np.array(y_train_all2)
y_train_all2 = -y_train_all2 # To flip the driving angle


In [ ]:
print(y_train_all2.shape)

In [ ]:
#y_train = np.concatenate((y_train3, y_train4), axis=0)
y_train = np.concatenate((y_train_all1, y_train_all2), axis=0)
y_train = np.array(y_train)

print('Y_train Shape', y_train.shape)

In [6]:
# Get image Data:

x_train1 = []

for i in range(len(centre_images1)):
    centre = imread(centre_images1[i], False, 'RGB')
    left   = imread(left_images1[i][1:], False, 'RGB')
    right  = imread(right_images1[i][1:], False, 'RGB')

    x_train1.append(centre)
    x_train1.append(left)
    x_train1.append(right)

In [7]:
x_train1 = np.array(x_train1)
print(x_train1.shape)

(22860, 160, 320, 3)


In [ ]:
# Get image Data:

x_train2 = []

for i in range(y_train2.shape[0]):
    centre = imread('data/'+ str(centre_images2[i]), False, 'RGB')
    left   = imread(os.path.join('data/', left_images2[i][1:]), False, 'RGB')
    right  = imread(os.path.join('data/', right_images2[i][1:]), False, 'RGB') 
    
    centre = np.fliplr(centre)
    left   = np.fliplr(left)
    right  = np.fliplr(right)
    
    x_train2.append(centre)
    x_train2.append(left)
    x_train2.append(right)

In [ ]:
x_train2 = np.array(x_train2)
print(x_train2.shape)

In [8]:
# Re-edit the images:
x_train_edit1 = []

for i in range(x_train1.shape[0]):
    read = x_train1[i, 60:,:,:]
    x_train_edit1.append(read)

In [9]:
x_train_edit1 = np.array(x_train_edit1)
print(x_train_edit1.shape)
del x_train1

(22860, 100, 320, 3)


In [ ]:
# Re-edit the images:
x_train_edit2 = []

for i in range(x_train2.shape[0]):
    read = x_train2[i, 60:,:,:]
    x_train_edit2.append(read)

In [ ]:
x_train_edit2 = np.array(x_train_edit2)
print(x_train_edit2.shape)
del x_train2

In [10]:
# Get image Data:
x_train_resize1 = []

for i in range(x_train_edit1.shape[0]):
    read = x_train_edit1[i]
    read = imresize(read, (66, 200), interp='bilinear')
    x_train_resize1.append(read)

In [11]:
x_train_resize1 = np.array(x_train_resize1)
print(x_train_resize1.shape)
del x_train_edit1

(22860, 66, 200, 3)


In [ ]:
x_train_resize2 = []

for i in range(x_train_edit2.shape[0]):
    read = x_train_edit2[i]
    read = imresize(read, (66, 200), interp='bilinear')
    x_train_resize2.append(read)

In [ ]:
x_train_resize2 = np.array(x_train_resize2)
print(x_train_resize2.shape)
del x_train_edit2

In [ ]:
x_train = []
x_train.extend(x_train_resize1)
x_train.extend(x_train_resize2)
x_train  = np.array(x_train)
print('X_train Shape', x_train.shape)

In [ ]:
# Delete variables to save memory

del x_train_resize1, x_train_resize2

In [ ]:
# Data:
print('X_train Shape', x_train.shape)
print('Y_train Shape', y_train.shape)

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')

ax1.imshow(x_train[4,:,:,:], cmap='gray')
ax1.set_title("Image Index:" + str(y_train[4]))

ax2.imshow(x_train[400,:,:,:], cmap='gray')
ax2.set_title("Image Index:" + str(y_train[400]))

ax3.imshow(x_train[3400,:,:,:], cmap='gray')
ax3.set_title("Image Index:" + str(y_train[3000]))

ax4.imshow(x_train[30035,:,:,:], cmap='gray')
ax4.set_title("Image Index:" + str(y_train[30035]))

In [10]:
def DNN():
    # Start
    model = Sequential()
    # set up lambda layer
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(66,200,3)))
    # Apply a 5x5 convolution with 24 output filters on a 31x98 image:
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode='valid', name='cnn1')) 
    # Add a 5x5 convolution on top, with 36 output filters:
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode='valid', name='cnn2'))
    # Add a 5x5 convolution on top, with 48 output filters:
    model.add(Convolution2D(54, 5, 5, subsample=(2, 2), border_mode='valid', name='cnn3'))
    # Add a 3x3 convolution on top, with 64 output filters:
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='valid', name='cnn4'))
    # Add a 3x3 convolution on top, with 64 output filters:
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='valid', name='cnn5'))
    # Flatten
    model.add(Flatten())
    # Fully-Connected Layer -1
    model.add(Dense(1164, name='fc1'))
    # Activation
    model.add(Activation('relu'))
    # Fully-Connected Layer -2
    model.add(Dense(100, name='fc2'))
    # Activation
    model.add(Activation('relu'))
    # Fully-Connected Layer -3
    model.add(Dense(50, name='fc3'))
    # Activation
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.5))
    # Fully-Connected Layer -4
    model.add(Dense(10, name='fc4'))
    # Activation
    model.add(Activation('relu'))
    # Output Layer
    model.add(Dense(1, name='fc5'))
    
    return model

In [11]:
model = DNN()
adm = adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adm, loss='mse')

In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cnn1 (Convolution2D)             (None, 31, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
cnn2 (Convolution2D)             (None, 14, 47, 36)    21636       cnn1[0][0]                       
____________________________________________________________________________________________________
cnn3 (Convolution2D)             (None, 5, 22, 54)     48654       cnn2[0][0]                       
___________________________________________________________________________________________

In [14]:
# Fit the model
#model.fit(x_train_resize, y_train, batch_size=256, nb_epoch=5, validation_split=0.01, shuffle=True)
#model.fit(x_train, y_train, batch_size=256, nb_epoch=5, validation_split=0.01, shuffle=True)
model.fit(x_train_resize1, y_train_all1, batch_size=256, nb_epoch=5, validation_split=0.01, shuffle=True)

json_string = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(json_string)

print("DNN architecture Saved,")

model.save_weights('model.h5')
print("Weights Saved.")

Train on 22631 samples, validate on 229 samples
Epoch 1/5
22631/22631 [==============================] - 96s - loss: 0.0587 - val_loss: 0.1488
Epoch 2/5
22631/22631 [==============================] - 96s - loss: 0.0512 - val_loss: 0.1088
Epoch 3/5
22631/22631 [==============================] - 96s - loss: 0.0486 - val_loss: 0.0845
Epoch 4/5
22631/22631 [==============================] - 96s - loss: 0.0471 - val_loss: 0.0858
Epoch 5/5
22631/22631 [==============================] - 96s - loss: 0.0465 - val_loss: 0.0806
DNN architecture Saved,
Weights Saved.
